# Notebook to plot Planet images and band histograms

Rainey Aberle

Spring 2022

In [ ]:
import os
import glob
import numpy as np
import rasterio as rio
import matplotlib.pyplot as plt

# path to Planet images
im_path = '/Users/raineyaberle/Research/PhD/Planet_snow_cover/study-sites/Gulkana/imagery/PlanetScope/adjusted-filtered/'
# output path (if saving figures)
out_path = im_path+'im_histograms/'
# determine whether to save output figures
save_figures = False # = True to save figures to out_path

In [ ]:
# -----Grab image file names
os.chdir(im_path) # change directory
im_names = glob.glob('*.tif') # load all .tif file names
im_names.sort() # sort file names by date

# -----Loop through images
for im_name in im_names:
    
    # load images
    im = rio.open(im_name)
    
    # load bands (blue, green, red, near infrared)
    b = im.read(1).astype(float) 
    g = im.read(2).astype(float)  
    r = im.read(3).astype(float) 
    nir = im.read(4).astype(float) 
    if np.nanmean(b) > 1e3:
        im_scalar = 10000
        b = b / im_scalar
        g = g / im_scalar
        r = r / im_scalar
        nir = nir / im_scalar
    # replace no data values with NaN
    b[b==0] = np.nan
    g[g==0] = np.nan
    r[r==0] = np.nan
    nir[nir==0] = np.nan
        
    # define grid
    im_x = np.linspace(im.bounds.left, im.bounds.right, num=np.shape(b)[1])
    im_y = np.linspace(im.bounds.top, im.bounds.bottom, num=np.shape(b)[0])
    
    # plot RGB image and band histograms
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,6), gridspec_kw={'height_ratios': [1]})
    plt.rcParams.update({'font.size': 12, 'font.serif': 'Arial'})
    ax1.imshow(np.dstack([r, g, b]), aspect='auto',
               extent=(np.min(im_x)/1000, np.max(im_x/1000), np.min(im_y)/1000, np.max(im_y)/1000))
    ax1.set_xlabel('Easting [km]')
    ax1.set_ylabel('Northing [km]')
    ax2.hist(b.flatten(), color='blue', histtype='step', bins=100, label='blue')
    ax2.hist(g.flatten(), color='green', histtype='step', bins=100, label='green')
    ax2.hist(r.flatten(), color='red', histtype='step', bins=100, label='red')
    ax2.hist(nir.flatten(), color='brown', histtype='step', linewidth=2, bins=100, label='NIR')
    ax2.set_xlabel('Surface reflectance')
    ax2.set_ylabel('Pixel counts')
    ax2.grid()
    ax2.legend(loc='right')
    fig.suptitle(im_name)
    fig.tight_layout()
    plt.show()
    
    # save figure to file
    if save_figures:
        fn = im_name[0:-4]+'_hist.png'
        fig.savefig(out_path+fn)
        print('figure saved to file')